# Test Chi-cuadrado

In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv('../dataset/base_datos_2008.csv')
df

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009723,2008,12,13,6,1002.0,959,1204.0,1150,DL,1636,...,6.0,45.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009724,2008,12,13,6,834.0,835,1021.0,1023,DL,1637,...,5.0,23.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009725,2008,12,13,6,655.0,700,856.0,856,DL,1638,...,24.0,12.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,...,13.0,13.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [24]:
# Generar una semilla permite replicar escenarios donde los resultados sean los mismos a pesar de la aleatoriedad de los datos
np.random.seed(0)

# Creamos un subset donde solo trabajemos con ciertos lugares 
# Houston, Atlanta, Indiana
df = df[ df['Origin'].isin(['HOU', 'ATL', 'IND'])]

# Ordenamos la base de datos
df = df.sample(frac = 1)

# Seleccionar solo las primeras 10000 filas
df = df[0 : 10000]

In [25]:
# Variable de valor verdadero cuando el retraso del vuelo es mayor a 30
df[ 'BigDelay'] = df[ 'ArrDelay'] > 30

# Tabla de contingencia que contiene las dos columnas que deseamos estudiar 
# Guardara tambien los margenes de la tabla
observados = pd.crosstab( index = df[ 'BigDelay'], columns = df['Origin'], margins = True)

In [26]:
# Mostramos lso valores de la tabla de contingencia
observados

# False - Numero de vuelos que se han retrasado poco y han salido de [Houston, Indiana, Atlanta]
# True - Numero de vuelos con retraso mayor a 30 y han salido de [Houston, Indiana, Atlanta]
# All -  Margen de suma por cada una de las columnas | - Margen de suma por cada una de las filas _

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,6927,883,765,8575
True,1197,129,99,1425
All,8124,1012,864,10000


In [27]:
# Importamos el modulo chi2_contingency que nos permite hacer el test

from scipy.stats import chi2_contingency

In [28]:
# Creamos un objeto a partir de la tabla observados

test =  chi2_contingency(observados)

In [29]:
test

# El primer valor es el estadistico (la suma de las diferencias al cuadrado.) 
# El segundo es el Pvalor para tomar desiciones en base a un criterio

#Luego la tabla de valores esperados (Que vamos a comparar von los valores observados)

(8.939538453043031,
 0.17700704816414425,
 6,
 array([[ 6966.33,   867.79,   740.88,  8575.  ],
        [ 1157.67,   144.21,   123.12,  1425.  ],
        [ 8124.  ,  1012.  ,   864.  , 10000.  ]]))

In [30]:
# Converitmos la tabla anterior en un dataframe para visualizar mejor los datos
esperados = pd.DataFrame(test[3])

In [31]:
# Los valores teoricos esperados no son del todo posibles ya que son decimales y solo es una aproximación teorica para comparar con los valores obtenidos
esperados

,0,1,2,3
0,6966.33,867.79,740.88,8575.0
1,1157.67,144.21,123.12,1425.0
2,8124.00,1012.00,864.00,10000.0


In [32]:
# Tabla con los valores esperados relativos
esperados_rel = round( esperados.apply( lambda r: r/len(df) * 100, axis = 1), 2)

# Redondeo a cada una de las tablas para poder aplicar la funcion lambda que es: Cada uno de los valores dividido por el total de valores multiplicado por 100 
# Asi obtenemos valores relativos y no absolutos (Que son mas dificiles de comparar)

In [33]:
# Tabla con los valores observador relativos
observados_rel = round( observados.apply( lambda r: r/len(df) * 100, axis = 1), 2)

In [35]:
observados_rel

Origin,ATL,HOU,IND,All
BigDelay,,,,
False,69.27,8.83,7.65,85.75
True,11.97,1.29,0.99,14.25
All,81.24,10.12,8.64,100.00


In [34]:
esperados_rel

,0,1,2,3
0,69.66,8.68,7.41,85.75
1,11.58,1.44,1.23,14.25
2,81.24,10.12,8.64,100.00


In [37]:
test[1]

# No hay relacion entre variables

0.17700704816414425

### Resumen del test de Hipotesis
- Si el p-valor < 0.05, hay diferencias significativas: Hay relacion entre variables
- Si el p-valor > 0.05, no hay diferencias significativas: No hay relacion entre variables